## ver10.1
### Use exponential of 'Jacquard' in weighted_graph.gexf as weight

In [1]:
# location of input files
graph_dir = 'C:/Users/U/Documents/BigData/'
#graph_dir = './'

In [2]:
### Set colaboratory True to run in Google Colaboratory. 
colaboratory = False

In [3]:
if colaboratory:
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [4]:
if colaboratory:
  # drive mean root directory of  google drive
  !mkdir -p drive
  !google-drive-ocamlfuse drive
  !ls drive/"Colab Notebooks"/Social_Proj/data


In [5]:
if colaboratory:
  # location of input files
  graph_dir = 'drive/Colab Notebooks/Social_Proj/data/'
  #graph_dir = './'

In [6]:
if colaboratory:
  !pip install python-louvain

In [7]:
import networkx as nx
import timeit
import community
#from networkx.algorithms.community import greedy_modularity_communities
import numpy as np
import pandas as pd
 
import operator
from timeit import default_timer as timer
from collections import OrderedDict
import os

In [8]:
pd.options.display.max_rows = 8

In [9]:
min_year = 1991
max_year = 2016
n_year=int(max_year - min_year + 1)


In [10]:
global_start = timer() ### Just for total calculation time measurement

In [11]:
#gephi_input_file = 'ml_graph expanded.gexf'
#gephi_input_file = 'ml_graph.gexf'
#gephi_input_file = 'ML Extended.gexf'
gephi_input_file = 'weighted_graph.gexf'
gephi_input = True

In [12]:
node_input = gephi_input_file
#node_input = 'ML articles expanded'
#node_input = 'ML articles'

In [13]:
if gephi_input:
    Graph = nx.read_gexf(graph_dir + gephi_input_file) 
    
    print('Gephi file read. Networkx class : ', type(Graph).__name__)
    if type(Graph).__name__ == 'DiGraph':
        Graph = Graph.to_undirected()
        print('Graph converted to: ', type(Graph).__name__)
    
    ML_df = pd.DataFrame([i[1] for i in Graph.nodes(data=True)], index=[i[0] for i in Graph.nodes(data=True)])
    ML_df.index.name = 'id'
    ML_df.reset_index(inplace=True)
    display(ML_df)

if not gephi_input:
    #ML_df = pd.read_csv(graph_dir + 'ML articles.csv')
    #ML_df = pd.read_csv(graph_dir + 'ML articles expanded.csv')
    ML_df = pd.read_csv(graph_dir + node_input + '.csv')
    #ML_df.loc[:,'year'] = ML_df.loc[:,'year'].apply(int)
    display(ML_df)

Gephi file read. Networkx class :  DiGraph
Graph converted to:  Graph


,id,Year,authors,label
0,261d674170c808873eea4b568089df1996c30ee0,2001,"[{'name': 'Daniel Rueckert', 'ids': ['1717710'...",261d674170c808873eea4b568089df1996c30ee0
1,c2414d375e18bd8d07da4a92268f7865bafd3ea5,2012,"[{'name': 'Fei Teng', 'ids': ['2928080']}, {'n...",c2414d375e18bd8d07da4a92268f7865bafd3ea5
2,e7b4a1d5d234d6a380ae400e0a9b86223dce362b,2012,"[{'name': 'Tao Chen', 'ids': ['2307631']}, {'n...",e7b4a1d5d234d6a380ae400e0a9b86223dce362b
3,a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7,2012,"[{'name': 'Tatjana Lange', 'ids': ['19141080']...",a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7
...,...,...,...,...
145219,328d5839774940f1c8c4fdd6dc36263d5528f8b8,1999,"[{'name': 'Sameer Singh', 'ids': ['3451035']}]",328d5839774940f1c8c4fdd6dc36263d5528f8b8
145220,4c6662ef67d9d7d369527175efaff2bbd1094e0a,2004,"[{'name': 'Nikola K. Kasabov', 'ids': ['168674...",4c6662ef67d9d7d369527175efaff2bbd1094e0a
145221,9c7b4ae23c6f8703fec225fd2d662611e16a115d,2001,"[{'name': 'Mohamad Obeid', 'ids': ['2126060']}...",9c7b4ae23c6f8703fec225fd2d662611e16a115d
145222,c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb,2006,"[{'name': 'Narendra K. Pareek', 'ids': ['34815...",c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb


In [14]:

cite_df = pd.read_csv(graph_dir + 'ML_cite_adjlist.csv')
cite_df.loc[:,'year'] = cite_df.loc[:,'year'].apply(int)
cite_df

,Unnamed: 0,from,to,year
0,0,008ea97d145202a469113171bfbfeb53b894b6cd,6a239ae3c8bb4aed7d545ff41f08be602ea96144,2012
1,1,008ea97d145202a469113171bfbfeb53b894b6cd,379472e4c1dc354ff392d54328e9bc9b1edc3a53,2012
2,2,008ea97d145202a469113171bfbfeb53b894b6cd,548c994134aab09054ea74fa0609acf8fb411f6b,2012
3,3,008ea97d145202a469113171bfbfeb53b894b6cd,f88370e1207c9ee5257a895f5b587db30148454b,2012
...,...,...,...,...
669803,669803,ffe3ca224357c1bba3192960c34fc1f5a5251b93,06865d55b7ceef648269eefa0c0dbb244fed9113,2010
669804,669804,ffe3ca224357c1bba3192960c34fc1f5a5251b93,7b3036df2eb0f12ba68d60ed946f56c6458feee8,2010
669805,669805,ffe3ca224357c1bba3192960c34fc1f5a5251b93,e626189ff3f46fac824b07b6506969c683aa067f,2010
669806,669806,ffe3ca224357c1bba3192960c34fc1f5a5251b93,8e452677804c1f601d90f1f8e935fc2b11baf5a4,2010


In [15]:

### Limit the citaiton from ML papers only
cite_df = pd.merge(cite_df, ML_df[['id']], left_on ='from', right_on = 'id', how = 'inner')
cite_df.loc[:,'year'] = cite_df.loc[:,'year'].apply(int)
cite_df

,Unnamed: 0,from,to,year,id
0,0,008ea97d145202a469113171bfbfeb53b894b6cd,6a239ae3c8bb4aed7d545ff41f08be602ea96144,2012,008ea97d145202a469113171bfbfeb53b894b6cd
1,1,008ea97d145202a469113171bfbfeb53b894b6cd,379472e4c1dc354ff392d54328e9bc9b1edc3a53,2012,008ea97d145202a469113171bfbfeb53b894b6cd
2,2,008ea97d145202a469113171bfbfeb53b894b6cd,548c994134aab09054ea74fa0609acf8fb411f6b,2012,008ea97d145202a469113171bfbfeb53b894b6cd
3,3,008ea97d145202a469113171bfbfeb53b894b6cd,f88370e1207c9ee5257a895f5b587db30148454b,2012,008ea97d145202a469113171bfbfeb53b894b6cd
...,...,...,...,...,...
594923,669803,ffe3ca224357c1bba3192960c34fc1f5a5251b93,06865d55b7ceef648269eefa0c0dbb244fed9113,2010,ffe3ca224357c1bba3192960c34fc1f5a5251b93
594924,669804,ffe3ca224357c1bba3192960c34fc1f5a5251b93,7b3036df2eb0f12ba68d60ed946f56c6458feee8,2010,ffe3ca224357c1bba3192960c34fc1f5a5251b93
594925,669805,ffe3ca224357c1bba3192960c34fc1f5a5251b93,e626189ff3f46fac824b07b6506969c683aa067f,2010,ffe3ca224357c1bba3192960c34fc1f5a5251b93
594926,669806,ffe3ca224357c1bba3192960c34fc1f5a5251b93,8e452677804c1f601d90f1f8e935fc2b11baf5a4,2010,ffe3ca224357c1bba3192960c34fc1f5a5251b93


In [16]:
ML_df.set_index('id', inplace = True)
dataset_df = ML_df.rename(index=str, columns={"year": "Year"})
dataset_df

,Year,authors,label
id,,,
261d674170c808873eea4b568089df1996c30ee0,2001,"[{'name': 'Daniel Rueckert', 'ids': ['1717710'...",261d674170c808873eea4b568089df1996c30ee0
c2414d375e18bd8d07da4a92268f7865bafd3ea5,2012,"[{'name': 'Fei Teng', 'ids': ['2928080']}, {'n...",c2414d375e18bd8d07da4a92268f7865bafd3ea5
e7b4a1d5d234d6a380ae400e0a9b86223dce362b,2012,"[{'name': 'Tao Chen', 'ids': ['2307631']}, {'n...",e7b4a1d5d234d6a380ae400e0a9b86223dce362b
a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7,2012,"[{'name': 'Tatjana Lange', 'ids': ['19141080']...",a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7
...,...,...,...
328d5839774940f1c8c4fdd6dc36263d5528f8b8,1999,"[{'name': 'Sameer Singh', 'ids': ['3451035']}]",328d5839774940f1c8c4fdd6dc36263d5528f8b8
4c6662ef67d9d7d369527175efaff2bbd1094e0a,2004,"[{'name': 'Nikola K. Kasabov', 'ids': ['168674...",4c6662ef67d9d7d369527175efaff2bbd1094e0a
9c7b4ae23c6f8703fec225fd2d662611e16a115d,2001,"[{'name': 'Mohamad Obeid', 'ids': ['2126060']}...",9c7b4ae23c6f8703fec225fd2d662611e16a115d
c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb,2006,"[{'name': 'Narendra K. Pareek', 'ids': ['34815...",c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb


In [17]:
if gephi_input:
    merged_df = dataset_df.copy()
    

if not gephi_input:
    

    
    oldest_in_cite_df = cite_df.groupby(['to'])[['year']].min()
    oldest_in_cite_df.index.name = 'id'
    print('# of all in-cited papers: ', len(oldest_in_cite_df))
    oldest_in_cite_df
    
    merged_df = pd.merge(dataset_df, oldest_in_cite_df, left_index = True, right_index = True, how = 'outer')
    #display(merged_df)
    #merged_df['Year'].combine_first(merged_df['year'])
    merged_df.loc[:, 'ML_flag'] = True
    merged_df.loc[pd.isnull(merged_df['Year']), 'ML_flag'] = False
    merged_df.loc[pd.isnull(merged_df['Year']), 'Year'] = merged_df.loc[pd.isnull(merged_df['Year']), 'year']
    merged_df.drop(['year'], axis=1, inplace = True)

    print(merged_df.shape)
    display(merged_df)


In [18]:
cite_df.loc[:,'Num_Out_Citations'] = 1

out_cite_df = cite_df.groupby(['from'])[['Num_Out_Citations']].sum()
out_cite_df.index.name = 'id'
out_cite_df

,Num_Out_Citations
id,
0003cef29c64620b7d962d20b6939b286124ef46,6
0003e735e0fbc2f2f571ce696c0a3d21ab57e614,5
0006204e4ba37eff6d926b2ef0b4d421b2158945,11
00076e4489cfda9a55932a1b32306cd51f40926a,36
...,...
fff73b4a46ac8d071155ffe30d708cf5cf3f2b53,9
fffae221f92229eeb398ba033040a8f283314c8b,5
fffd458edd60ec0a19603fe8436df5c9d12d26ae,24
fffea404de3b99a2c601f2139c88524e43a5213e,81


In [19]:

merged1_df = pd.merge(merged_df, out_cite_df, left_index = True, right_index = True,  how = 'left')
merged1_df.loc[:,'Num_Out_Citations'].fillna(0, inplace = True)

if gephi_input:
    cols = ['Year', 'authors', 'journalName', 'title', 'paperAbstract'] + ['Num_Out_Citations']
if not gephi_input:
    cols = ['Year', 'authors', 'journalName', 'title', 'paperAbstract'] + ['keyPhrases', 'venue', 'pdfUrls', 's2Url', 'ML_flag'] + ['Num_Out_Citations']
merged1_df = merged1_df.loc[:, cols]
print(merged_df.shape)
merged1_df

(145223, 3)


C:\Users\U\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Year,authors,journalName,title,paperAbstract,Num_Out_Citations
id,,,,,,
261d674170c808873eea4b568089df1996c30ee0,2001,"[{'name': 'Daniel Rueckert', 'ids': ['1717710'...",NaN,NaN,NaN,0.0
c2414d375e18bd8d07da4a92268f7865bafd3ea5,2012,"[{'name': 'Fei Teng', 'ids': ['2928080']}, {'n...",NaN,NaN,NaN,50.0
e7b4a1d5d234d6a380ae400e0a9b86223dce362b,2012,"[{'name': 'Tao Chen', 'ids': ['2307631']}, {'n...",NaN,NaN,NaN,0.0
a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7,2012,"[{'name': 'Tatjana Lange', 'ids': ['19141080']...",NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
328d5839774940f1c8c4fdd6dc36263d5528f8b8,1999,"[{'name': 'Sameer Singh', 'ids': ['3451035']}]",NaN,NaN,NaN,0.0
4c6662ef67d9d7d369527175efaff2bbd1094e0a,2004,"[{'name': 'Nikola K. Kasabov', 'ids': ['168674...",NaN,NaN,NaN,0.0
9c7b4ae23c6f8703fec225fd2d662611e16a115d,2001,"[{'name': 'Mohamad Obeid', 'ids': ['2126060']}...",NaN,NaN,NaN,0.0
c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb,2006,"[{'name': 'Narendra K. Pareek', 'ids': ['34815...",NaN,NaN,NaN,0.0


In [20]:
merged2_df = merged1_df[merged1_df['Year'].notnull()].copy()
merged2_df.loc[:,:].fillna('_', inplace = True)

print(merged2_df.shape)
merged2_df

(145223, 6)


,Year,authors,journalName,title,paperAbstract,Num_Out_Citations
id,,,,,,
261d674170c808873eea4b568089df1996c30ee0,2001,"[{'name': 'Daniel Rueckert', 'ids': ['1717710'...",_,_,_,0.0
c2414d375e18bd8d07da4a92268f7865bafd3ea5,2012,"[{'name': 'Fei Teng', 'ids': ['2928080']}, {'n...",_,_,_,50.0
e7b4a1d5d234d6a380ae400e0a9b86223dce362b,2012,"[{'name': 'Tao Chen', 'ids': ['2307631']}, {'n...",_,_,_,0.0
a7c2af21f0b94bb66273f9ae28e4cd8e9a04ecf7,2012,"[{'name': 'Tatjana Lange', 'ids': ['19141080']...",_,_,_,0.0
...,...,...,...,...,...,...
328d5839774940f1c8c4fdd6dc36263d5528f8b8,1999,"[{'name': 'Sameer Singh', 'ids': ['3451035']}]",_,_,_,0.0
4c6662ef67d9d7d369527175efaff2bbd1094e0a,2004,"[{'name': 'Nikola K. Kasabov', 'ids': ['168674...",_,_,_,0.0
9c7b4ae23c6f8703fec225fd2d662611e16a115d,2001,"[{'name': 'Mohamad Obeid', 'ids': ['2126060']}...",_,_,_,0.0
c2074f2d3f03b30b042f7f8187ec34c2f0eb79eb,2006,"[{'name': 'Narendra K. Pareek', 'ids': ['34815...",_,_,_,0.0


In [21]:
### merged2_df.to_csv(graph_dir + 'All_Papers' '.csv')

In [22]:
if not gephi_input:
    ### Create graph

    Graph = nx.Graph()

    ### Add edges and edge attributes (year)
    cite_a_list_df = cite_df

    adj=cite_a_list_df[['from','to']]
    adj_tuple=list(adj.itertuples(index=False,name=None))   
    Graph.add_edges_from(adj_tuple)
    #nx.write_gexf(Graph, graph_dir + 'ML+_graph.gexf') ## To check numpy float/int error

In [23]:
if not gephi_input:
    cite_a_list_df['from_to'] = cite_a_list_df[['from', 'to']].apply(tuple, axis=1)
    edge_year_dict=dict(zip(cite_a_list_df['from_to'], [int(year) for year in cite_a_list_df['year']]))
    nx.set_edge_attributes(Graph, edge_year_dict,name='Year')
    #nx.write_gexf(Graph, graph_dir + 'ML+_graph.gexf') ## To check numpy float/int error

In [24]:

for col in cols:    
    node_dict = merged2_df[col].to_dict()
    if col in ['Year', 'Num_Out_Citations']:
        node_dict  = {k:int(v) for (k, v) in node_dict.items()}
    if col in ['ML_flag']:
        node_dict  = {k:bool(v) for (k, v) in node_dict.items()}
    nx.set_node_attributes(Graph, node_dict, col)

### Write to Gephi files
#nx.write_gexf(Graph, graph_dir + 'ML+_graph.gexf') ## To check numpy float/int error

In [25]:
jac=nx.get_edge_attributes(Graph,'Jacquard')
jac_exp={k:float(np.exp(5*v)) for (k,v) in jac.items()}
nx.set_edge_attributes(Graph,jac_exp,'Jacquard_Exp' )
### Write to Gephi files
#nx.write_gexf(Graph, graph_dir + 'ML+_graph.gexf') ## To check numpy float/int error

In [26]:
num_nodes = len(Graph.node)
num_edges = Graph.number_of_edges()
print('# of nodes : ', num_nodes, '| # of edges : ', num_edges)

# of nodes :  145223 | # of edges :  981146


### Extract the largest connected component

In [27]:
Graph = max(nx.connected_component_subgraphs(Graph), key=len)

In [28]:
# add degree and Num_In_Citations later
degree_dict = dict(Graph.degree())
nx.set_node_attributes(Graph, degree_dict, 'Degree')
num_out_citations_dict = nx.get_node_attributes(Graph, 'Num_Out_Citations' )

num_in_citations_dict = {i: (int(degree_dict[i] - num_out_citations_dict[i])) for i in Graph.node}
#num_in_citations_dict = dict([[i, (int(degree_dict[i] - num_out_citations_dict[i]))] for i in Graph.node])
#num_in_citations_dict  = {}
#for i in Graph.node:
#    num_in_citations_dict[i] = int(degree_dict[i] - num_out_citations_dict[i])

nx.set_node_attributes(Graph,num_in_citations_dict,'Num_In_Citations')

In [29]:
# Here we create subgraphs, nodes and edges are accumulated every year. Calculation done for CNM and Louvain method

def subgraph_by_year(Graph,option='accumulate',connected='yes'):
    """
    option='accumulate' will accumulate the nodes and edges of the graph year on year
    option='separate' will only keep the nodes year on year, edges from previous years will not be retained
    connected='yes' will only use the largest connected component for each year
    connected='no' will use all available nodes for each year
    """
    
    # get node and edge year
    node_yr=nx.get_node_attributes(Graph,'Year')
    edge_yr=nx.get_edge_attributes(Graph,'Year')
    
    # dictionarys to filter nodes and edges by year
    n_year=int(max(node_yr.values())-min(node_yr.values()))+1
    min_year=min(node_yr.values())
    list_dict_node_year=[{} for i in range(n_year)]
    list_dict_edge_year=[{} for i in range(n_year)]
    for i in range(n_year):
        if option=='accumulate':
            list_dict_edge_year[i]={k:v for (k,v) in edge_yr.items() if  v<=min_year+i}
            list_dict_node_year[i]={k:v for (k,v) in node_yr.items() if  v<=min_year+i}
        elif option=='separate':
            list_dict_edge_year[i]={k:v for (k,v) in edge_yr.items() if  v==min_year+i}
            list_dict_node_year[i]={k:v for (k,v) in node_yr.items() if  v<=min_year+i}       
        
        else:
            raise Exception("wrong keyword for option. use accumulate or separate only")
     
    print('Input Graph has',Graph.number_of_nodes(),'nodes and',Graph.number_of_edges(),'edges')  
    H=[nx.Graph() for i in range(n_year)]
    if option=='accumulate':
        for i in range(n_year):
            start = timer()
            if connected=='no':
                 H[i]=nx.subgraph(Graph,list(list_dict_node_year[i].keys())).copy()
            elif connected=='yes':
                 H[i]=nx.subgraph(Graph,list(list_dict_node_year[i].keys())).copy()
                 H[i]=max(nx.connected_component_subgraphs(H[i]), key=len)
            #H[i].add_nodes_from(list(list_dict_node_year[i].keys()))
            #H[i].add_edges_from(list(list_dict_edge_year[i].keys()))
            else:
                raise Exception("wrong keyword for connected. use yes or no only")
            end = timer()    
            print('Calculation Time (sec): ', "{0: 6.0f}".format(end - start), '|' + 'Year:',str(i+min_year),'--',H[i].number_of_nodes(),'nodes --',H[i].number_of_edges(),'edges')

    elif option=='separate':
        for i in range(n_year):
            start = timer()
            if connected=='no':
                 H[i]=nx.subgraph(Graph,list(list_dict_node_year[i].keys())).copy()
            elif connected=='yes':
                 H[i]=nx.subgraph(Graph,list(list_dict_node_year[i].keys())).copy()
                 H[i]=max(nx.connected_component_subgraphs(H[i]), key=len)
            #H[i].add_nodes_from(list(list_dict_node_year[i].keys()))
            #H[i].add_edges_from(list(list_dict_edge_year[i].keys()))
            else:
                raise Exception("wrong keyword for connected. use yes or no only")            
            
            end = timer()    
            print('Calculation Time (sec): ', "{0:.0f}".format(end - start), '|' + 'Year:',str(i+min_year),'--',H[i].number_of_nodes(),'nodes --',H[i].number_of_edges(),'edges')
            

    return H

In [30]:
connected = 'no' ### connected='yes' option used to take too long.

In [31]:
Graphs = subgraph_by_year(Graph,option='accumulate',connected=connected) 

Input Graph has 144463 nodes and 980629 edges
Calculation Time (sec):       1 |Year: 1991 -- 1022 nodes -- 164 edges
Calculation Time (sec):       1 |Year: 1992 -- 2375 nodes -- 847 edges
Calculation Time (sec):       0 |Year: 1993 -- 4196 nodes -- 2372 edges
Calculation Time (sec):       0 |Year: 1994 -- 6668 nodes -- 5885 edges
Calculation Time (sec):       0 |Year: 1995 -- 9827 nodes -- 12021 edges
Calculation Time (sec):       0 |Year: 1996 -- 13580 nodes -- 21538 edges
Calculation Time (sec):       1 |Year: 1997 -- 17981 nodes -- 34609 edges
Calculation Time (sec):       1 |Year: 1998 -- 23242 nodes -- 52734 edges
Calculation Time (sec):       1 |Year: 1999 -- 29100 nodes -- 76909 edges
Calculation Time (sec):       2 |Year: 2000 -- 35601 nodes -- 105437 edges
Calculation Time (sec):       2 |Year: 2001 -- 42563 nodes -- 139328 edges
Calculation Time (sec):       2 |Year: 2002 -- 50239 nodes -- 180108 edges
Calculation Time (sec):       3 |Year: 2003 -- 59122 nodes -- 233678 edges

In [32]:
### Add Degree and Num_In_Citations which may increase every year 

for j in range(n_year):
    Graph = Graphs[j]
    
    # add degree and Num_In_Citations later
    degree_dict = dict(Graph.degree())
    nx.set_node_attributes(Graph, degree_dict, 'Degree')
    num_out_citations_dict = nx.get_node_attributes(Graph, 'Num_Out_Citations' )

    num_in_citations_dict = {i: (int(degree_dict[i] - num_out_citations_dict[i])) for i in Graph.node}
    #num_in_citations_dict = dict([[i, (int(degree_dict[i] - num_out_citations_dict[i]))] for i in Graph.node])
    #num_in_citations_dict  = {}
    #for i in Graph.node:
    #    num_in_citations_dict[i] = int(degree_dict[i] - num_out_citations_dict[i])

    nx.set_node_attributes(Graph,num_in_citations_dict,'Num_In_Citations')


In [33]:
### Write to Gephi files
#for i in range(len(Graphs)):
for i in range(1):
    nx.write_gexf(Graphs[i], graph_dir + 'ML_sub_' + node_input + '_' + str(i+min_year) + ".gexf")

### Cluster graph

In [34]:
try:
    Graphs
except NameError:
    ### Read Gephi files
    Graphs = [[] for i in range(n_year)]  
    for i in range(n_year):
        Graphs[i]=nx.read_gexf(graph_dir + 'ML_sub_' + node_input + '_' + str(i+min_year) + ".gexf")
else:
    print('Graphs are already in memory.')



Graphs are already in memory.


In [35]:
def cluster_graph(H, resolution, weight = 'weight'):
    print("------------Louvain------------------")
    results_df = pd.DataFrame()
    results_df.index.name = 'Timepoint'
    for i in range(n_year):
        start = timeit.default_timer()      
        if i == 0:
            cluster_dict = {} 
        num_clusters_last = len(set(cluster_dict.values()))
        Graph = H[i]
        num_nodes = len(Graph.nodes)
        partition_dict = {}
        num_increment = 0
        for node in Graph.nodes:
            if node in cluster_dict:
                partition_dict[node] = cluster_dict[node]
            else: 
                partition_dict[node] = num_clusters_last + num_increment
                num_increment += 1
        cluster_dict = community.best_partition(Graph, resolution=resolution, partition = partition_dict, weight = weight)
        num_clusters = len(set(cluster_dict.values()))
        nx.set_node_attributes(Graph, cluster_dict,'Louvain cluster')

        stop = timeit.default_timer()
        cal_time = stop-start
        
        num_edges = len(Graph.edges)
        modularity = community.modularity(cluster_dict,Graph)
        
        results_df.loc[(i+min_year), 'Resolution'] = resolution
        results_df.loc[(i+min_year), 'Num_Clusters'] = num_clusters
        results_df.loc[(i+min_year), 'Modularity'] = modularity
        results_df.loc[(i+min_year), 'Num_Nodes'] = num_nodes
        results_df.loc[(i+min_year), 'Num_Edges'] = num_edges
        results_df.loc[(i+min_year), 'Calculation_Time'] = cal_time
        #display(results_df.loc[(i+min_year):(i+min_year+1),:])
        print('Year: {:4d}'.format(i+min_year), "| {:6d} nodes ".format(num_nodes),"| {: 5d} clusters".format(num_clusters),"| Modularity: {:.6f}".format(modularity), " | Calculation time: {: 6.2f} sec".format(cal_time))  
    
    #w = pd.ExcelWriter(graph_dir + 'Clustering_Results' + desc + '.xlsx')
    #sheetname = 'Clustering_Results'
    #results_df.to_excel(w, sheetname)
    #w.sheets[sheetname].set_column(0, 7, 10)
    #w.save()
    results_df.to_csv(graph_dir + 'Clustering_Results_ver10' + desc + '.csv')
    display(results_df)
    
    return H

In [36]:
resolution = 2.2
#weight = 'weight'
weight = 'Jacquard_Exp'
#weight = 'Year_Diff'

desc = '_{0:s}_connected{1:s}_res{2:.2f}_wtBy{3:s}_initpart'.format(node_input, connected, resolution, weight) 
print(desc)

Clustered=cluster_graph(Graphs, resolution=resolution, weight=weight) 

_weighted_graph.gexf_connectedno_res2.20_wtByJacquard_Exp_initpart
------------Louvain------------------
Year: 1991 |   1022 nodes  |   867 clusters | Modularity: 0.959697  | Calculation time:   0.09 sec
Year: 1992 |   2375 nodes  |  1649 clusters | Modularity: 0.966830  | Calculation time:   0.24 sec
Year: 1993 |   4196 nodes  |  2317 clusters | Modularity: 0.941275  | Calculation time:   0.51 sec
Year: 1994 |   6668 nodes  |  2748 clusters | Modularity: 0.887710  | Calculation time:   0.80 sec
Year: 1995 |   9827 nodes  |  3110 clusters | Modularity: 0.842927  | Calculation time:   1.01 sec
Year: 1996 |  13580 nodes  |  3475 clusters | Modularity: 0.819941  | Calculation time:   1.40 sec
Year: 1997 |  17981 nodes  |  3806 clusters | Modularity: 0.796911  | Calculation time:   1.94 sec
Year: 1998 |  23242 nodes  |  4070 clusters | Modularity: 0.781993  | Calculation time:   2.87 sec
Year: 1999 |  29100 nodes  |  4879 clusters | Modularity: 0.751809  | Calculation time:   4.77 sec
Year

,Resolution,Num_Clusters,Modularity,Num_Nodes,Num_Edges,Calculation_Time
Timepoint,,,,,,
1991,2.2,867.0,0.959697,1022.0,164.0,0.093429
1992,2.2,1649.0,0.966830,2375.0,847.0,0.244186
1993,2.2,2317.0,0.941275,4196.0,2372.0,0.510444
1994,2.2,2748.0,0.887710,6668.0,5885.0,0.804419
...,...,...,...,...,...,...
2013,2.2,5491.0,0.663972,135260.0,863499.0,61.883717
2014,2.2,4918.0,0.661586,139518.0,913082.0,50.647371
2015,2.2,4311.0,0.660065,142453.0,950556.0,52.339656
2016,2.2,3486.0,0.658925,144463.0,980629.0,37.592375


### Write to Gephi files
for i in range(len(Graphs)):
    nx.write_gexf(Graphs[i], graph_dir + 'ML_clustered_' + node_input + '_' + str(i+min_year) + ".gexf")

### Calculate z and P

In [37]:
#Copy_Clus2 = Clustered.copy()
Copy_Clus2 = [g.copy() for g in Clustered.copy()]

In [38]:
try:
    Graphs
except NameError:
    ### Read Gephi files
    Graphs = [[] for i in range(n_year)]  
    for i in range(n_year):
        Graphs[i]=nx.read_gexf(graph_dir + 'ML_clustered_' + node_input + '_' + str(i+min_year) + ".gexf")
else:
    print('Graphs are already in memory.')



Graphs are already in memory.


In [39]:
### Calculate P, z, and cluster size by Pandas DataFrame manipulation



#for Graph in Graphs[0:1]:
def calculate_z_p(Graph,clus_attr = 'Louvain cluster'):
    start = timer()
    
    ### Calculate P value by Pandas 
    edges = [[u,v] for (u,v) in Graph.edges()]
    n0_df = pd.DataFrame.from_records(edges, columns=['Node', 'Node Connected'])
    n1_df = pd.DataFrame.from_records(edges, columns=['Node Connected', 'Node'])
    n2_df = pd.concat([n0_df, n1_df], ignore_index = True)
    n2_df.set_index('Node Connected', inplace=True)
    #display(n2_df)
    num_edges = n2_df.shape[0]
                                   
    c_df = pd.DataFrame(pd.Series(nx.get_node_attributes(Graph,clus_attr), name='Cluster of Node Connected'))
    c_df.index.name = 'Node Connected'
    #display(c_df)
    num_nodes = c_df.shape[0]
    
    
    n3_df = pd.merge(n2_df, c_df, left_index=True, right_index=True)
    #display(n3_df)
    n3_df.loc[:,'Degree per Cluster'] = 1
    n4_df = n3_df.groupby(['Node','Cluster of Node Connected'])[['Degree per Cluster']].sum()
    n4_df.reset_index(inplace=True)
    n4_df.set_index('Node', inplace=True)
    #display(n4_df)
    
    d_df = pd.DataFrame(pd.Series(nx.get_node_attributes(Graph,'Degree'), name='Degree'))
    d_df.index.name = 'Node'
    #display(d_df)
    
    n5_df = pd.merge(n4_df, d_df, left_index=True, right_index=True, how='right')
    n5_df['Degree per Cluster'].fillna(0, inplace=True)
    n5_df.loc[:,'Degree positive'] = n5_df['Degree']
    n5_df.loc[n5_df['Degree'] == 0,'Degree_positive'] = 1 ### Avoid zero that will be enominator
    n5_df.loc[:,'Sq Ratio of Degree per Cluster'] = n5_df[['Degree per Cluster', 'Degree positive']].apply(lambda x: (x[0] / x[1]) ** 2, axis=1)
    #display(n5_df)
    
    n6_df = n5_df.reset_index().groupby('Node')[['Sq Ratio of Degree per Cluster']].sum()
    n6_df.loc[:,'P'] = 1 - n6_df['Sq Ratio of Degree per Cluster']
    #display(n6_df)
    
    P_dict_np = n6_df['P'].to_dict()
    #display(P_dict_np.items())
    P_dict = dict([k,float(v)] for (k,v) in P_dict_np.items())
    #display(P_dict)
    
    nx.set_node_attributes(Graph,P_dict,'P')
    
    ### Calculate z
    
    #display(n3_df)
    z4_df = n3_df.reset_index()
    z4_df.set_index('Node', inplace=True)
    #display(z4_df)
    
    c2_df = c_df.rename(columns={'Cluster of Node Connected':'Cluster of Node'})
    c2_df.index.name = 'Node'
    #display(c2_df)
    z5_df = pd.merge(z4_df, c2_df, left_index=True, right_index=True)
    z5_df.loc[:,'Within Cluster Flag'] = z5_df[['Cluster of Node', 'Cluster of Node Connected']].apply(lambda x: x[0] == x[1], axis=1)
    #display(z5_df)
    #display(z5_df.groupby(['Within Cluster Flag']).count())
    z6_df = z5_df.loc[z5_df['Within Cluster Flag'] == True,:]
    z6_df.reset_index(inplace=True)
    #display(z6_df)
    z7_df = z6_df.groupby(['Node','Cluster of Node'])[['Degree per Cluster']].sum()
    #display(z7_df)
    #z7_2_df = z7_df.copy()
    #z7_2_df.loc[:,'Degree per Cluster Positive'] = z7_2_df['Degree per Cluster']
    #z7_2_df = z7_2_df.groupby(['Cluster of Node'])[['Degree per Cluster Positive']].transform(lambda x: x.std())
    #z7_2_df.loc[z7_2_df['Degree per Cluster Positive'] == 0,'Degree per Cluster Positive'] = 1 ### Avoid zero that will be enominator 
    #z7_df.loc[:,'Degree per Cluster Positive'] = z7_2_df['Degree per Cluster Positive']
    #z8_df = z7_df.groupby(['Cluster of Node'])[['Degree per Cluster', 'Degree per Cluster Positive']].transform(lambda x: ((x[0] - x[0].mean()) / x[1]), axis=1)
    z8_df = z7_df.groupby(['Cluster of Node'])[['Degree per Cluster']].transform(lambda x: ((x - x.mean()) / x.std()))
    z8_df.reset_index(inplace=True)
    z8_df.set_index('Node', inplace=True)
    #display(z8_df)
    z_dict_np = z8_df['Degree per Cluster'].to_dict()
    z_dict = dict([k,float(v)] for (k,v) in z_dict_np.items())
    #display(z_dict)
    nx.set_node_attributes(Graph,z_dict,'z')
    
    ##### Calculate correct P value (20180412)
    P8_df = z7_df.reset_index()
    P8_df.set_index('Node', inplace=True)
    display(P8_df)
    P9_df = pd.merge(P8_df, d_df, left_index=True, right_index=True, how='right')
    display(P9_df)
    #P9
    
    #####
    
    ### Calculate cluster size
    #display(c_df)
    #c2_df = c_df.reset_index()
    c2_df = c_df.copy()
    c2_df.loc[:,'Cluster Size'] = 1
    #display(c2_df)
    s_df = c2_df.groupby(['Cluster of Node Connected'])[['Cluster Size']].transform(sum)
    #display(s_df)
    csize_dict_np = s_df['Cluster Size'].to_dict()
    csize_dict = dict([k,int(v)] for (k,v) in csize_dict_np.items())
    #display(csize_dict)
    nx.set_node_attributes(Graph,csize_dict,'Cluster_Size')
    
    end = timer()
    print("#{0: 7d} nodes".format(num_nodes), " | {0: 7d} edges".format(num_edges), ' | Time for calculation of z and P: {0:.0f} sec'.format(end - start))

In [40]:
def get_oldest_paper(Graph,clus_attr):
    clusters=nx.get_node_attributes(Graph,clus_attr)
    n_clus=len(set(clusters.values()))
    
    seed_df=pd.DataFrame()
    for i in range(n_clus):
        c= {k for (k,v) in clusters.items() if  v==i}
        H=nx.subgraph(Graph,c)
        old=min(nx.get_node_attributes(H,'Year').values())
        paper_id=[k for (k,v) in nx.get_node_attributes(H,'Year').items() if v==old]       
        
        J=nx.subgraph(H,paper_id)
        
        df =  pd.DataFrame([i[1] for i in J.nodes(data=True)], index=[i[0] for i in J.nodes(data=True)])
        df.loc[:,'cluster'] = i
        df.index.name = 'id'
        df.reset_index(inplace=True)
        df.index.name = 'index'
        df = df.loc[:,['Year','cluster','cluster_size','id','authors','journalName','title','paperAbstract','Num_Out_Citations','degree','Num_In_Citations', 'z','P']]
        #display(df)

        seed_df=seed_df.append(other=df)
    return seed_df
 

def write_oldest_papers(seed_df, year):
    
    seed_df.sort_values(by = ['Year', 'cluster'], ascending = False, inplace = True)
    seed_df.reset_index(drop=True, inplace=True)
    seed_df.index.name = 'index'
    #seed_df = seed_df.drop(['id'],axis=1)

    #seed_df.to_csv(graph_dir + 'oldest_paper_' + str(i+min_year) '.csv') 
    w = pd.ExcelWriter(graph_dir + 'Oldest_Papers_' + str(year) + '.xlsx')
    sheetname = 'Oldest_Papers'
    seed_df.to_excel(w, sheetname)
    w.sheets[sheetname].set_column(7, 8, 50)
    w.sheets[sheetname].autofilter(0,0,seed_df.shape[0] + 1, seed_df.shape[1] + 1)
    w.save()

def write_all_papers(Graph, year):
    ML_df = pd.DataFrame([i[1] for i in Graph.nodes(data=True)], index=[i[0] for i in Graph.nodes(data=True)])
    ML_df.index.name = 'id'
    ML_df.reset_index(inplace=True)
    ML_df.index.name = 'index'
    ML_df.to_csv(graph_dir + 'All_Papers_' + str(year) + '.csv')

In [41]:
start = timer()
df_list = []
for year in range(min_year, max_year+1):
    print('[Processing year:' + str(year) + ']')
    Graph = Copy_Clus2[year-min_year]
    calculate_z_p(Graph,'Louvain cluster')
    nodes = Graph.nodes(data=True)
    df = pd.DataFrame([i[1] for i in nodes], index=[i[0] for i in nodes])
    df.index.name = 'id'
    df.reset_index(inplace=True)
    df.loc[:, 'Timepoint'] = year
    #print(df.columns)
    df_list.append(df[['id', 'Timepoint', 'Louvain cluster', 'Cluster_Size', 'z', 'P', 'Degree', 'Num_In_Citations']])

end = timer()
print('\n[Finished to calculate z and P for all years] \n Total calculation time: {0:.0f} sec'.format(end - start))

[Processing year:1991]


C:\Users\U\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  del sys.path[0]
C:\Users\U\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in double_scalars


,Cluster of Node,Degree per Cluster
Node,,
015242aa19ec5a82a4bd9373e0924b7256c5c553,502,2
025bd24869ec8c8c66f0a00715785f9320fc1c30,202,1
02cc8ee37b11e7bd96a3bc1095a8bee6b154bcf4,127,2
031e3c7055c9ad2f3d3ba3aa1c472fc584b052fc,127,1
...,...,...
f88804e245ae4fcc8379afab6d6fe7c405208fd5,141,1
fa9d10b8c6b2645ab2c49cb6c7370bed23061272,160,1
fc801b6119fe0fe3fa3f7e6f79d439949224f7d0,219,1
fd2a430a6ca5a0678cef56b44236c359f36ba6ad,261,1


,Cluster of Node,Degree per Cluster,Degree
Node,,,
c1a6a4f03ed2c1d9326310c86aba0cff6dcb3040,NaN,NaN,0
de2ac61afa5e4a224b5b3b30df7a6cb3288b99ca,1.0,1.0,1
b703ccf490b0cf0395961334e06d30c4dae5e657,NaN,NaN,0
2310dd0b2a1689eb9e2d1ea7ccdc165303719246,NaN,NaN,0
...,...,...,...
b2d57093d0dd486cd26548cfd5e29e9cbb08e4f7,NaN,NaN,0
4228de4b132a1df2d9740c1f22c6fe785dc35b6c,NaN,NaN,0
cb4ddff658d0260e4533cd4b2f31f27741017f60,115.0,1.0,1
451f292801b9767a0b989a891ef8ee5caa875ad2,NaN,NaN,0


#   1022 nodes  |     328 edges  | Time for calculation of z and P: 0 sec
[Processing year:1992]


...

,Cluster of Node,Degree per Cluster,Degree
Node,,,
574e7fca47990fb394647b4161c54fac68b78d85,0.0,11.0,13
497e94ef1d1b08c34c6005b0fef6203733e586ba,1.0,2.0,4
134bb1cfe1cd0d37ef1cf091105a4e5a37241898,2.0,4.0,10
5224b500544e561fe038aa69630dfd0bc256ab78,3.0,18.0,34
...,...,...,...
664490d768024067215477b8274b6ac5529513f7,4.0,17.0,19
728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,16.0,16.0,18
66c65b758b1a55e75b17bef7b74924b42ebff4c6,0.0,4.0,7
6778fd27fad5f2314d63622da4b80a80772e271c,4.0,4.0,4


# 142453 nodes  |  1901112 edges  | Time for calculation of z and P: 67 sec
[Processing year:2016]


,Cluster of Node,Degree per Cluster
Node,,
00000088bbc15a03ab89d8da6c356bf25aea9519,642,2
0000a09bb47d436b675937186ab27b753bf7e0e5,5,8
0000d0d159ad3d9702ad82d83a3e20e7fc6637cb,2,10
0000d7919aabb85fc11a83719d99f4b5c6f21bac,6,2
...,...,...
fffea404de3b99a2c601f2139c88524e43a5213e,5,21
fffeb86c30c05131bdc71785c57762a68841114e,8,7
ffff2cd797317607e23af945dcd34cb9fe46967c,5,11
ffff61c00e3be793980e43a562bf70d43e321493,5,8


,Cluster of Node,Degree per Cluster,Degree
Node,,,
574e7fca47990fb394647b4161c54fac68b78d85,0,11,13
497e94ef1d1b08c34c6005b0fef6203733e586ba,1,2,4
134bb1cfe1cd0d37ef1cf091105a4e5a37241898,2,4,10
5224b500544e561fe038aa69630dfd0bc256ab78,3,18,34
...,...,...,...
664490d768024067215477b8274b6ac5529513f7,4,17,19
728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,15,16,18
66c65b758b1a55e75b17bef7b74924b42ebff4c6,0,4,7
6778fd27fad5f2314d63622da4b80a80772e271c,4,4,4


# 144463 nodes  |  1961258 edges  | Time for calculation of z and P: 66 sec

[Finished to calculate z and P for all years] 
 Total calculation time: 748 sec


In [42]:
stack_df = pd.concat(df_list) 
stack_df.sort_values(['id', 'Timepoint'], inplace=True)
stack_df.reset_index(drop=True, inplace=True)
stack_df.index.name = 'index'
display(stack_df)

,id,Timepoint,Louvain cluster,Cluster_Size,z,P,Degree,Num_In_Citations
index,,,,,,,,
0,00000088bbc15a03ab89d8da6c356bf25aea9519,2005,958,1,NaN,1.0,0,0
1,00000088bbc15a03ab89d8da6c356bf25aea9519,2006,1039,1,NaN,1.0,0,0
2,00000088bbc15a03ab89d8da6c356bf25aea9519,2007,597,1,NaN,1.0,0,0
3,00000088bbc15a03ab89d8da6c356bf25aea9519,2008,627,1,NaN,1.0,0,0
...,...,...,...,...,...,...,...,...
1762170,ffff61c00e3be793980e43a562bf70d43e321493,2013,5,19114,-0.258414,0.0,8,8
1762171,ffff61c00e3be793980e43a562bf70d43e321493,2014,5,19775,-0.264420,0.0,8,8
1762172,ffff61c00e3be793980e43a562bf70d43e321493,2015,5,20155,-0.271280,0.0,8,8
1762173,ffff61c00e3be793980e43a562bf70d43e321493,2016,5,20552,-0.276812,0.0,8,8


In [43]:
#stack_df.to_csv(graph_dir + 'All_Papers_Timepoint_Stack' + desc + '.csv', encoding='utf-8')

In [44]:
stack_df.set_index(['id', 'Timepoint'])

Louvain cluster  \
id                                       Timepoint                    
00000088bbc15a03ab89d8da6c356bf25aea9519 2005                   958   
                                         2006                  1039   
                                         2007                   597   
                                         2008                   627   
...                                                             ...   
ffff61c00e3be793980e43a562bf70d43e321493 2013                     5   
                                         2014                     5   
                                         2015                     5   
                                         2016                     5   

                                                    Cluster_Size         z  \
id                                       Timepoint                           
00000088bbc15a03ab89d8da6c356bf25aea9519 2005                  1       NaN   
                                         2006                  1       NaN   
                                         2007                  1       NaN   
                                         2008                  1       NaN   
...                                                          ...       ...   
ffff61c00e3be793980e43a562bf70d43e321493 2013              19114 -0.258414   
                                         2014              19775 -0.264420   
                                         2015              20155 -0.271280   
                                         2016              20552 -0.276812   

                                                      P  Degree  \
id                                       Timepoint                
00000088bbc15a03ab89d8da6c356bf25aea9519 2005       1.0       0   
                                         2006       1.0       0   
                                         2007       1.0       0   
                                         2008       1.0       0   
...                                                 ...     ...   
ffff61c00e3be793980e43a562bf70d43e321493 2013       0.0       8   
                                         2014       0.0       8   
                                         2015       0.0       8   
                                         2016       0.0       8   

                                                    Num_In_Citations  
id                                       Timepoint                    
00000088bbc15a03ab89d8da6c356bf25aea9519 2005                      0  
                                         2006                      0  
                                         2007                      0  
                                         2008                      0  
...                                                              ...  
ffff61c00e3be793980e43a562bf70d43e321493 2013                      8  
                                         2014                      8  
                                         2015                      8  
                                         2016                      8  

[1762174 rows x 6 columns]

In [45]:
output_unstack = True

In [46]:
if output_unstack:
    unstack_df = stack_df.set_index(['id', 'Timepoint']).unstack()
    unstack_df

In [47]:
if output_unstack:
    unstack_df.columns = ['{0[0]:s} {0[1]:4d}'.format(col) for col in unstack_df.columns]
    unstack_df.index.name = 'id'
    display(unstack_df)


,Louvain cluster 1991,Louvain cluster 1992,Louvain cluster 1993,Louvain cluster 1994,Louvain cluster 1995,Louvain cluster 1996,Louvain cluster 1997,Louvain cluster 1998,Louvain cluster 1999,Louvain cluster 2000,...,Num_In_Citations 2007,Num_In_Citations 2008,Num_In_Citations 2009,Num_In_Citations 2010,Num_In_Citations 2011,Num_In_Citations 2012,Num_In_Citations 2013,Num_In_Citations 2014,Num_In_Citations 2015,Num_In_Citations 2016
id,,,,,,,,,,,,,,,,,,,,,
00000088bbc15a03ab89d8da6c356bf25aea9519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0000a09bb47d436b675937186ab27b753bf7e0e5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.0,7.0,8.0,8.0,8.0
0000d0d159ad3d9702ad82d83a3e20e7fc6637cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,11.0,11.0
0000d7919aabb85fc11a83719d99f4b5c6f21bac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffea404de3b99a2c601f2139c88524e43a5213e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-40.0
fffeb86c30c05131bdc71785c57762a68841114e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,...,4.0,4.0,4.0,4.0,4.0,6.0,6.0,7.0,7.0,7.0
ffff2cd797317607e23af945dcd34cb9fe46967c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,9.0,11.0,11.0,12.0,12.0
ffff61c00e3be793980e43a562bf70d43e321493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,6.0,6.0,6.0,7.0,7.0,8.0,8.0,8.0,8.0


In [48]:
if output_unstack:
    unstack_w_df = unstack_df.reset_index()
    display(unstack_w_df)
    unstack_w_df.index.name = 'index'
    #unstack_w_df.to_csv(graph_dir + 'All_Papers_Timepoint_Untack' + desc + '.csv', encoding='utf-8')

,id,Louvain cluster 1991,Louvain cluster 1992,Louvain cluster 1993,Louvain cluster 1994,Louvain cluster 1995,Louvain cluster 1996,Louvain cluster 1997,Louvain cluster 1998,Louvain cluster 1999,...,Num_In_Citations 2007,Num_In_Citations 2008,Num_In_Citations 2009,Num_In_Citations 2010,Num_In_Citations 2011,Num_In_Citations 2012,Num_In_Citations 2013,Num_In_Citations 2014,Num_In_Citations 2015,Num_In_Citations 2016
0,00000088bbc15a03ab89d8da6c356bf25aea9519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,0000a09bb47d436b675937186ab27b753bf7e0e5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,7.0,7.0,8.0,8.0,8.0
2,0000d0d159ad3d9702ad82d83a3e20e7fc6637cb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,10.0,10.0,10.0,11.0,11.0,11.0,11.0,11.0,11.0
3,0000d7919aabb85fc11a83719d99f4b5c6f21bac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144459,fffea404de3b99a2c601f2139c88524e43a5213e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-40.0
144460,fffeb86c30c05131bdc71785c57762a68841114e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,4.0,6.0,6.0,7.0,7.0,7.0
144461,ffff2cd797317607e23af945dcd34cb9fe46967c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,7.0,9.0,11.0,11.0,12.0,12.0
144462,ffff61c00e3be793980e43a562bf70d43e321493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,6.0,6.0,6.0,7.0,7.0,8.0,8.0,8.0,8.0


In [49]:
Graph = Copy_Clus2[-1]
nodes = Graph.nodes(data=True)
df = pd.DataFrame([i[1] for i in nodes], index=[i[0] for i in nodes])
df.index.name = 'id'



In [50]:
### Add Oldest in cluster flag

df.loc[:,'Oldest_in_Cluster Year'] = df.groupby(['Louvain cluster'])['Year'].transform(min)
df.loc[:,'Oldest_in_Cluster Flag'] = df[['Year', 'Oldest_in_Cluster Year']].apply(lambda x: x[0] == x[1], axis=1)
df

,Cluster_Size,Degree,Louvain cluster,Num_In_Citations,Num_Out_Citations,P,Year,authors,journalName,label,paperAbstract,title,z,Oldest_in_Cluster Year,Oldest_in_Cluster Flag
id,,,,,,,,,,,,,,,
574e7fca47990fb394647b4161c54fac68b78d85,13667,13,0,-5,18,0.260355,2012,"[{'name': 'Qiulei Ding', 'ids': ['7335329']}, ...",_,574e7fca47990fb394647b4161c54fac68b78d85,_,_,0.021775,1991,False
497e94ef1d1b08c34c6005b0fef6203733e586ba,8,4,1,4,0,0.625000,2007,"[{'name': 'James D. Ralston', 'ids': ['2560902...",_,497e94ef1d1b08c34c6005b0fef6203733e586ba,_,_,0.353553,2002,False
134bb1cfe1cd0d37ef1cf091105a4e5a37241898,12110,10,2,10,0,0.660000,2004,"[{'name': 'Raphaël Marée', 'ids': ['1689882']}...",_,134bb1cfe1cd0d37ef1cf091105a4e5a37241898,_,_,-0.459832,1991,False
5224b500544e561fe038aa69630dfd0bc256ab78,4151,34,3,-6,40,0.570934,2009,"[{'name': 'Abdul-Rahman Mawlood-Yunis', 'ids':...",_,5224b500544e561fe038aa69630dfd0bc256ab78,_,_,1.422177,1991,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664490d768024067215477b8274b6ac5529513f7,23276,19,4,19,0,0.193906,2013,"[{'name': 'Elena Bellodi', 'ids': ['2372439']}...",_,664490d768024067215477b8274b6ac5529513f7,_,_,0.267139,1991,False
728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,4082,18,15,-1,19,0.197531,2010,"[{'name': 'Martin Homola', 'ids': ['3304771']}...",_,728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,_,_,0.710666,1991,False
66c65b758b1a55e75b17bef7b74924b42ebff4c6,13667,7,0,7,0,0.612245,1996,"[{'name': 'L.-M. Reissell', 'ids': ['2066867']}]",_,66c65b758b1a55e75b17bef7b74924b42ebff4c6,_,_,-0.442497,1991,False
6778fd27fad5f2314d63622da4b80a80772e271c,23276,4,4,4,0,0.000000,2000,"[{'name': 'Lawrence J. Mazlack', 'ids': ['1688...",_,6778fd27fad5f2314d63622da4b80a80772e271c,_,_,-0.551489,1991,False


In [51]:
cols = [col for col in ['ML_flag', 'Oldest_in_Cluster Flag', 'Oldest_in_Cluster Year', 'Year', 'authors', 'journalName', 'venue', 'pdfUrls', 's2Url', 'keyPhrases', 'title', 'paperAbstract', 'Num_Out_Citations'] if col in df.columns.tolist()]
cols

['Oldest_in_Cluster Flag',
 'Oldest_in_Cluster Year',
 'Year',
 'authors',
 'journalName',
 'title',
 'paperAbstract',
 'Num_Out_Citations']

In [52]:
static_df = df[cols].copy()

In [53]:
if output_unstack:
    complete_df = pd.merge(static_df, unstack_df, left_index = True, right_index = True)
    display(complete_df)
    complete_w_df = complete_df.reset_index()
    complete_w_df.index.name = 'index'
    complete_w_df.to_csv(graph_dir + 'All_Papers_Complete_ver10' + desc + '.csv', encoding='utf-8')

,Oldest_in_Cluster Flag,Oldest_in_Cluster Year,Year,authors,journalName,title,paperAbstract,Num_Out_Citations,Louvain cluster 1991,Louvain cluster 1992,...,Num_In_Citations 2007,Num_In_Citations 2008,Num_In_Citations 2009,Num_In_Citations 2010,Num_In_Citations 2011,Num_In_Citations 2012,Num_In_Citations 2013,Num_In_Citations 2014,Num_In_Citations 2015,Num_In_Citations 2016
id,,,,,,,,,,,,,,,,,,,,,
574e7fca47990fb394647b4161c54fac68b78d85,False,1991,2012,"[{'name': 'Qiulei Ding', 'ids': ['7335329']}, ...",_,_,_,18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-5.0,-5.0,-5.0,-5.0,-5.0
497e94ef1d1b08c34c6005b0fef6203733e586ba,False,2002,2007,"[{'name': 'James D. Ralston', 'ids': ['2560902...",_,_,_,0,NaN,NaN,...,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
134bb1cfe1cd0d37ef1cf091105a4e5a37241898,False,1991,2004,"[{'name': 'Raphaël Marée', 'ids': ['1689882']}...",_,_,_,0,NaN,NaN,...,9.0,9.0,9.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0
5224b500544e561fe038aa69630dfd0bc256ab78,False,1991,2009,"[{'name': 'Abdul-Rahman Mawlood-Yunis', 'ids':...",_,_,_,40,NaN,NaN,...,NaN,NaN,-9.0,-7.0,-6.0,-6.0,-6.0,-6.0,-6.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664490d768024067215477b8274b6ac5529513f7,False,1991,2013,"[{'name': 'Elena Bellodi', 'ids': ['2372439']}...",_,_,_,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,18.0,19.0,19.0
728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,False,1991,2010,"[{'name': 'Martin Homola', 'ids': ['3304771']}...",_,_,_,19,NaN,NaN,...,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
66c65b758b1a55e75b17bef7b74924b42ebff4c6,False,1991,1996,"[{'name': 'L.-M. Reissell', 'ids': ['2066867']}]",_,_,_,0,NaN,NaN,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
6778fd27fad5f2314d63622da4b80a80772e271c,False,1991,2000,"[{'name': 'Lawrence J. Mazlack', 'ids': ['1688...",_,_,_,0,NaN,NaN,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0


In [54]:
measures = ['Cluster_Size', 'z', 'P', 'Degree', 'Num_In_Citations']

In [55]:
transform_dict = {col: {col+'_diff':'diff', col+'_shift':'shift'} for col in measures}
display(transform_dict)
stack_diff_df = stack_df.groupby('id').agg(transform_dict)
stack_diff_df

{'Cluster_Size': {'Cluster_Size_diff': 'diff', 'Cluster_Size_shift': 'shift'},
 'Degree': {'Degree_diff': 'diff', 'Degree_shift': 'shift'},
 'Num_In_Citations': {'Num_In_Citations_diff': 'diff',
  'Num_In_Citations_shift': 'shift'},
 'P': {'P_diff': 'diff', 'P_shift': 'shift'},
 'z': {'z_diff': 'diff', 'z_shift': 'shift'}}

C:\Users\U\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


Cluster_Size                            z                P  \
        Cluster_Size_diff Cluster_Size_shift    z_diff   z_shift P_diff   
index                                                                     
0                     NaN                NaN       NaN       NaN    NaN   
1                     0.0                1.0       NaN       NaN    0.0   
2                     0.0                1.0       NaN       NaN    0.0   
3                     0.0                1.0       NaN       NaN    0.0   
...                   ...                ...       ...       ...    ...   
1762170             880.0            18234.0  0.059558 -0.317972    0.0   
1762171             661.0            19114.0 -0.006006 -0.258414    0.0   
1762172             380.0            19775.0 -0.006860 -0.264420    0.0   
1762173             397.0            20155.0 -0.005532 -0.271280    0.0   

                     Degree                   Num_In_Citations  \
        P_shift Degree_diff Degree_shift Num_In_Citations_diff   
index                                                            
0           NaN         NaN          NaN                   NaN   
1           1.0         0.0          0.0                   0.0   
2           1.0         0.0          0.0                   0.0   
3           1.0         0.0          0.0                   0.0   
...         ...         ...          ...                   ...   
1762170     0.0         1.0          7.0                   1.0   
1762171     0.0         0.0          8.0                   0.0   
1762172     0.0         0.0          8.0                   0.0   
1762173     0.0         0.0          8.0                   0.0   

                                
        Num_In_Citations_shift  
index                           
0                          NaN  
1                          0.0  
2                          0.0  
3                          0.0  
...                        ...  
1762170                    7.0  
1762171                    8.0  
1762172                    8.0  
1762173                    8.0  

[1762174 rows x 10 columns]

In [56]:
stack_diff_df.columns = ['{0[1]:s}'.format(col) for col in stack_diff_df.columns]
stack_diff_df.index.name = 'index'
display(stack_diff_df)

,Cluster_Size_diff,Cluster_Size_shift,z_diff,z_shift,P_diff,P_shift,Degree_diff,Degree_shift,Num_In_Citations_diff,Num_In_Citations_shift
index,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1762170,880.0,18234.0,0.059558,-0.317972,0.0,0.0,1.0,7.0,1.0,7.0
1762171,661.0,19114.0,-0.006006,-0.258414,0.0,0.0,0.0,8.0,0.0,8.0
1762172,380.0,19775.0,-0.006860,-0.264420,0.0,0.0,0.0,8.0,0.0,8.0
1762173,397.0,20155.0,-0.005532,-0.271280,0.0,0.0,0.0,8.0,0.0,8.0


In [57]:
#stack2_df = pd.merge(stack_df, stack_diff_df, left_index = True, right_index = True)
#stack2_df

In [58]:
stack2_df = stack_df.copy()
for measure in measures:
    stack2_df.loc[:,measure+'(d)'] = stack_diff_df.loc[:,measure+'_diff'] / stack_diff_df.loc[:,measure+'_shift']
stack2_df
    

,id,Timepoint,Louvain cluster,Cluster_Size,z,P,Degree,Num_In_Citations,Cluster_Size(d),z(d),P(d),Degree(d),Num_In_Citations(d)
index,,,,,,,,,,,,,
0,00000088bbc15a03ab89d8da6c356bf25aea9519,2005,958,1,NaN,1.0,0,0,NaN,NaN,NaN,NaN,NaN
1,00000088bbc15a03ab89d8da6c356bf25aea9519,2006,1039,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
2,00000088bbc15a03ab89d8da6c356bf25aea9519,2007,597,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
3,00000088bbc15a03ab89d8da6c356bf25aea9519,2008,627,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762170,ffff61c00e3be793980e43a562bf70d43e321493,2013,5,19114,-0.258414,0.0,8,8,0.048261,-0.187306,NaN,0.142857,0.142857
1762171,ffff61c00e3be793980e43a562bf70d43e321493,2014,5,19775,-0.264420,0.0,8,8,0.034582,0.023243,NaN,0.000000,0.000000
1762172,ffff61c00e3be793980e43a562bf70d43e321493,2015,5,20155,-0.271280,0.0,8,8,0.019216,0.025942,NaN,0.000000,0.000000
1762173,ffff61c00e3be793980e43a562bf70d43e321493,2016,5,20552,-0.276812,0.0,8,8,0.019697,0.020392,NaN,0.000000,0.000000


In [59]:
stack2_df = stack2_df.reset_index(drop=True)
stack2_df.set_index('id', inplace=True)
stack2_df

,Timepoint,Louvain cluster,Cluster_Size,z,P,Degree,Num_In_Citations,Cluster_Size(d),z(d),P(d),Degree(d),Num_In_Citations(d)
id,,,,,,,,,,,,
00000088bbc15a03ab89d8da6c356bf25aea9519,2005,958,1,NaN,1.0,0,0,NaN,NaN,NaN,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,2006,1039,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,2007,597,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,2008,627,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
ffff61c00e3be793980e43a562bf70d43e321493,2013,5,19114,-0.258414,0.0,8,8,0.048261,-0.187306,NaN,0.142857,0.142857
ffff61c00e3be793980e43a562bf70d43e321493,2014,5,19775,-0.264420,0.0,8,8,0.034582,0.023243,NaN,0.000000,0.000000
ffff61c00e3be793980e43a562bf70d43e321493,2015,5,20155,-0.271280,0.0,8,8,0.019216,0.025942,NaN,0.000000,0.000000
ffff61c00e3be793980e43a562bf70d43e321493,2016,5,20552,-0.276812,0.0,8,8,0.019697,0.020392,NaN,0.000000,0.000000


In [60]:
static2_df = df[['Oldest_in_Cluster Flag', 'Oldest_in_Cluster Year', 'Year', 'title', 'Num_Out_Citations']].copy()
static2_df

,Oldest_in_Cluster Flag,Oldest_in_Cluster Year,Year,title,Num_Out_Citations
id,,,,,
574e7fca47990fb394647b4161c54fac68b78d85,False,1991,2012,_,18
497e94ef1d1b08c34c6005b0fef6203733e586ba,False,2002,2007,_,0
134bb1cfe1cd0d37ef1cf091105a4e5a37241898,False,1991,2004,_,0
5224b500544e561fe038aa69630dfd0bc256ab78,False,1991,2009,_,40
...,...,...,...,...,...
664490d768024067215477b8274b6ac5529513f7,False,1991,2013,_,0
728db6d2e8ef6c328d2081d9c6a70821ab03a4b6,False,1991,2010,_,19
66c65b758b1a55e75b17bef7b74924b42ebff4c6,False,1991,1996,_,0
6778fd27fad5f2314d63622da4b80a80772e271c,False,1991,2000,_,0


In [61]:
long_df = pd.merge(static2_df, stack2_df, left_index = True, right_index = True)
display(long_df)
long_w_df = long_df.reset_index()
long_w_df.index.name = 'index'
long_w_df.to_csv(graph_dir + 'All_Papers_Long_ver10' + desc + '.csv', encoding='utf-8')



,Oldest_in_Cluster Flag,Oldest_in_Cluster Year,Year,title,Num_Out_Citations,Timepoint,Louvain cluster,Cluster_Size,z,P,Degree,Num_In_Citations,Cluster_Size(d),z(d),P(d),Degree(d),Num_In_Citations(d)
id,,,,,,,,,,,,,,,,,
00000088bbc15a03ab89d8da6c356bf25aea9519,False,1997,2005,_,0,2005,958,1,NaN,1.0,0,0,NaN,NaN,NaN,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,False,1997,2005,_,0,2006,1039,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,False,1997,2005,_,0,2007,597,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
00000088bbc15a03ab89d8da6c356bf25aea9519,False,1997,2005,_,0,2008,627,1,NaN,1.0,0,0,0.000000,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff61c00e3be793980e43a562bf70d43e321493,False,1991,2002,_,0,2013,5,19114,-0.258414,0.0,8,8,0.048261,-0.187306,NaN,0.142857,0.142857
ffff61c00e3be793980e43a562bf70d43e321493,False,1991,2002,_,0,2014,5,19775,-0.264420,0.0,8,8,0.034582,0.023243,NaN,0.000000,0.000000
ffff61c00e3be793980e43a562bf70d43e321493,False,1991,2002,_,0,2015,5,20155,-0.271280,0.0,8,8,0.019216,0.025942,NaN,0.000000,0.000000
ffff61c00e3be793980e43a562bf70d43e321493,False,1991,2002,_,0,2016,5,20552,-0.276812,0.0,8,8,0.019697,0.020392,NaN,0.000000,0.000000


In [62]:
long2_df = long_df.copy()
long2_df.groupby('Cluster_Size')[['Louvain cluster']].nunique()

,Louvain cluster
Cluster_Size,
1,6702
2,5742
3,4369
4,3044
...,...
21888,1
22533,1
22992,1
23276,1


In [63]:
start = timer()
nx.write_gexf(Graph, graph_dir + 'ML_zP_ver10_' + str(max_year) + desc + '.gexf')
end = timer()
print('Time to write Gephi file: {0:.0f} sec'.format(end - start)) 

Time to write Gephi file: 115 sec


In [64]:
global_end = timer()
print('Time for all the processings: {0:.0f} sec'.format(global_end - global_start)) 

Time for all the processings: 1890 sec
